In [1]:
# @title Install libraries
!pip install sentence_transformers transformers langchain einops
!pip install accelerate bitsandbytes trl peft loralib
!pip install chromadb faiss-gpu pacmap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 2.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl 

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# @title Load data

from io import BytesIO
from IPython.display import display

import requests
import pandas as pd

gg_id = '814552626'
file_id = '11Xrvp1vdB0UJ34CIM_A-hKAnCJp_0BQA0HF6GGTerRc'
file_path = f'https://docs.google.com/spreadsheets/d/{file_id}/export?format=csv&gid={gg_id}'

df = pd.read_csv(file_path)
df = df.ffill()
df['Id'] = df.reset_index(drop=False)['index'].astype(str).str.zfill(5)
df

,Object,Detail,Interpretation,Id
0,Ảnh,Đốt ảnh,Tình duyên sắp thành,00000
1,Ảnh,Tặng ảnh,Điềm chia ly / sắp đi xa,00001
2,Ảnh,Được tặng ảnh,Điềm chia ly / sắp đi xa,00002
3,Ảnh,Ảnh người yêu,Có tình địch,00003
4,Ảnh,Xé ảnh người khác,Sắp hội ngộ người đó,00004
...,...,...,...,...
2303,Xưởng,Trông nom xưởng thở,Tài lộc hanh thông,02303
2304,Xưởng,Đi ra khỏi xưởng,Bạn sẽ được quý nhân cất nhắc,02304
2305,Xưởng,Đi ra khỏi xưởng rồi quay lại,Bạn đang được chủ tin cậy,02305
2306,Yêu,Yêu tha thiết 1 người mà người ấy luôn ruồng rẫy,Hôn nhân sắp thành,02306


In [4]:
# @title Load language model with Quantization
import torch
from transformers import pipeline
from transformers import AutoConfig, BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
# )

# model_name = "vinai/PhoGPT-7B5-Instruct"
model_name = "vinai/PhoGPT-4B-Chat"
config = AutoConfig.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, config=config, load_in_8bit=True)
model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

configuration_mpt.py:   0%|          | 0.00/16.4k [00:00<?, ?B/s]

blocks.py:   0%|          | 0.00/4.04k [00:00<?, ?B/s]

ffn.py:   0%|          | 0.00/5.22k [00:00<?, ?B/s]

...

meta_init_context.py:   0%|          | 0.00/3.96k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vinai/PhoGPT-4B-Chat:
- meta_init_context.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/vinai/PhoGPT-4B-Chat:
- modeling_mpt.py
- custom_embedding.py
- adapt_tokenizer.py
- param_init_fns.py
- hf_prefixlm_converter.py
- meta_init_context.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


pytorch_model.bin:   0%|          | 0.00/7.38G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/91.0 [00:00<?, ?B/s]

MPTForCausalLM(
  (transformer): MPTModel(
    (wte): SharedEmbedding(20480, 3072)
    (emb_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0-31): 32 x MPTBlock(
        (norm_1): LPLayerNorm((3072,), eps=1e-05, elementwise_affine=True)
        (attn): MultiheadAttention(
          (Wqkv): Linear8bitLt(in_features=3072, out_features=9216, bias=True)
          (out_proj): Linear8bitLt(in_features=3072, out_features=3072, bias=True)
        )
        (norm_2): LPLayerNorm((3072,), eps=1e-05, elementwise_affine=True)
        (ffn): MPTMLP(
          (up_proj): Linear8bitLt(in_features=3072, out_features=12288, bias=True)
          (down_proj): Linear8bitLt(in_features=12288, out_features=3072, bias=True)
        )
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_ffn_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (norm_f): LPLayerNorm((3072,), eps=1e-05, elementwise_affine=True)
  )
)

In [5]:
# @title Apply LoRA
from peft import AutoPeftModelForCausalLM, PeftModel

model_path = "/content/drive/MyDrive/Colab Notebooks/Checkpoints/PhoGPT-4B-Chat-LoRA-dreamerpretation-v1"
model_lora = PeftModel.from_pretrained(model, model_path)
model_lora

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MPTForCausalLM(
      (transformer): MPTModel(
        (wte): SharedEmbedding(20480, 3072)
        (emb_drop): Dropout(p=0.0, inplace=False)
        (blocks): ModuleList(
          (0-31): 32 x MPTBlock(
            (norm_1): LPLayerNorm((3072,), eps=1e-05, elementwise_affine=True)
            (attn): MultiheadAttention(
              (Wqkv): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=3072, out_features=9216, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=9216, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding

In [21]:
# @title Answer Generation
# Reference: https://community.openai.com/t/cheat-sheet-mastering-temperature-and-top-p-in-chatgpt-api/172683

def generate_answer(symbol):

    # Generate prompt
    sample = [
        {"role": "user", "content": f"Ý nghĩa khi mơ thấy {symbol.lower()}",},
    ]
    prompt = tokenizer.apply_chat_template(sample, tokenize=False, add_generation_prompt=True)

    # Generate response
    input_ids = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
                    inputs = input_ids["input_ids"].to("cuda"),
            attention_mask = input_ids["attention_mask"].to("cuda"),
                do_sample = True,
            temperature = 0.169,
        repetition_penalty = 1.1,
                    top_k = 50,
                    top_p = 0.11,
            max_new_tokens = 128,
            eos_token_id = tokenizer.eos_token_id,
            pad_token_id = tokenizer.pad_token_id,
    )

    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    response = response.split("### Trả lời:")[-1]
    return response


for index, row in df.sample(n=10).iterrows():
    response = generate_answer(row['Detail'])

    print('\n', index, row['Detail'])
    print('\t', row['Interpretation'])
    print('\t', response)

/root/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/116013fa63f8c4025739487e1cbff65b7375bbe2/attention.py:87: UserWarning: Propagating key_padding_mask to the attention module and applying it within the attention module can cause unnecessary computation/memory usage. Consider integrating into attn_bias once and passing that to each attention module instead.
  warnings.warn('Propagating key_padding_mask to the attention module ' + 'and applying it within the attention module can cause ' + 'unnecessary computation/memory usage. Consider integrating ' + 'into attn_bias once and passing that to each attention ' + 'module instead.')



 392 Xỏ chỉ vào kim
	 Sắp gặp rắc rối trong công việc
	  Sắp gặp rắc rối trong công việc

 1136 Sống trong làng
	 Gia đình êm ấm
	  Gia đình êm ấm

 1897 Vú tự nhiên chảy sữa
	 Bạn đã phải lòng 1 người
	  Bạn đã phải lòng 1 người

 1925 Khiêng quan tài trong đám ma
	 Sẽ có người mang đến lợi lộc
	  Sẽ có người mang đến lợi lộc

 711 Bị chặt đầu
	 Hạnh phúc sắp tới bên mình
	  Hạnh phúc sắp tới bên mình

 2307 Bị người yêu xỉ nhục
	 Đang có người mơ ước cưới được bạn
	  Đang có người mơ ước cưới được bạn

 1567 Trang sức bằng bạc
	 Tình yêu sẽ đạt thành tựu
	  Tình yêu sẽ đạt thành tựu

 1424 Nhổ nấm
	 Sắp nhận được của bất ngờ
	  Sắp nhận được của bất ngờ

 189 Thấy người khác đang bơi
	 Điềm may
	  Điềm may

 74 Ăn thịt trâu
	 Sắp gặp chuyện tốt lành
	  Sắp gặp chuyện tốt lành


In [22]:
# @title Answer Generation with Complexity

df['group'] = df.index / 2.55
df['group'] = df['group'].sample(frac=1).values.astype(int)

df_mixed = df.groupby(['group']).agg({
                'Id':  'count',
            'Detail': ', '.join,
    'Interpretation': '. '.join,
})

for complexity in [2, 3]:
    for index, row in df_mixed[df_mixed['Id'] == complexity].sample(n=7).iterrows():
        response = generate_answer(row['Detail'])

        print('\n', index, row['Detail'])
        print('\t', row['Interpretation'])
        print('\t', response)

/root/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/116013fa63f8c4025739487e1cbff65b7375bbe2/attention.py:87: UserWarning: Propagating key_padding_mask to the attention module and applying it within the attention module can cause unnecessary computation/memory usage. Consider integrating into attn_bias once and passing that to each attention module instead.
  warnings.warn('Propagating key_padding_mask to the attention module ' + 'and applying it within the attention module can cause ' + 'unnecessary computation/memory usage. Consider integrating ' + 'into attn_bias once and passing that to each attention ' + 'module instead.')



 537 Được tặng dê đực, Cháy rừng
	 Sắp hưởng gia tài lớn. Việc làm ăn thịnh vượng
	  Có nguy cơ cãi vã với hàng xóm

 604 Được người khác tặng hoa hồng, Đốt thuốc lá
	 Bạn sẽ thành công vang dội trong lĩnh vực giao tiếp. Sắp có người thích bạn, mời bạn đi ăn uống
	  Bạn sẽ thành công trong tình trường

 773 Nghe tiếng ó kêu, Người tàn tật đến nhà
	 Lợi lộc sẽ liên tiếp vào nhà. Sắp có được 1 số tiền nhỏ
	  Sẽ có kẻ giàu sang đến nhà

 2 Ném bùn vào người khác, Cầm gậy
	 Sắp rửa sạch hàm oan. Sắp gặp chuyện phiền muộn
	  Sắp rửa sạch hàm oan

 42 Đánh bạc, Bị bề trên trách phạt
	 Bạn sắp có cơ hội tốt, cần nắm bắt. Sắp được khen thưởng
	  Điềm báo đỗ đạt

 895 Bầy gà vịt vây quanh, Ngồi trên ghế
	 Sắp cãi vã với hàng xóm. Sắp có người giỏi giang hỗ trợ trong công việc
	  Sắp có chuyện cãi vã với hàng xóm

 822 Đêm thanh vắng, Nghe tiếng kèn thúc giục
	 Gia đình hạnh phúc. Bạn sẽ làm hỏng 1 cơ hội tốt
	  Bạn sẽ nhận được tin của người ở xa

 403 Người khác đưa chổi, Râu tự rụng, Thằn lằ